This notebook is the optimization of the algorithms for Mikania. Here, the data passes through the feature creation pipeline, goes to the ml training pipeline and gridsearch so that the best algorithms with the best parameters are chosen. 

No preliminary testing was done here as the data was not available yet. Also, previous data from mikania were obtained in positive and negative ionization

# Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.svm import SVC
from sklearn import metrics

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.model_selection import PredefinedSplit


# Functions

In [2]:
# creates the feature name with the mz and rt

def feature_name_creation(xcms_file_path):
    table = pd.read_csv(xcms_file_path, index_col=[0]) 
    
    # no need for decimal on m/z (low resolution) and only one decimal for rt
    table.mz = table.mz.round(0).astype(int)
    table.rt = table.rt.round(1)

    # creating the feature name: mz_rt
    features = table["mz"].astype(str) + "_" + table["rt"].astype(str)
    table.insert(0, 'features', features) # first column
    
    # drop as we don't know how many columns the table will have. Drop the known ones. 
    # There should only be the 'features' column and the samples
    
    table_clean = table.drop(['isotopes', 'adduct','pcgroup'], axis=1) #'npeaks','NEG_GROUP', 'POS_GROUP',
    
    return table_clean

In [3]:
# rounds the mz and rt columns along with its min and max

def rounder(dataframe):
    table = dataframe 
    
    table.mz = table.mz.round(0).astype(int)
    table.mzmin = table.mzmin.round(0).astype(int)
    table.mzmax = table.mzmax.round(0).astype(int)
    
    table.rt = table.rt.round(1)
    table.rtmin = table.rtmin.round(1)
    table.rtmax = table.rtmax.round(1)

    
    return table

# Data Prep Pipeline

`Train` and `Val` sets were processed separately on `xcms` - excludes the possibility of data leakage 
But, when processing is separated, the features can be slightly different. The compounds are almost the same, but due to processing steps, there can be shifts on the decimals of `mz` or `rt`. 
For this reason, creating the feature name concatenating `mz_rt` on train and val might not produce the same features, and machine learning training is not possible with that. 

Errors observed in this case are related to the fact that features observed in train were not present in validation and vice versa or the order of the features were different in both datasets. This 'pipeline' fixes this issue.

**Steps:**

1. Creates the name for the features on `Train` set - this is the set used as reference. Whatever features where observed here, should appear on `Val`. The name is created concatenating `mz` and `rt` columns (`mz_rt`)
2. Creates a correspondance between the feature on `Train` and `Val` set, giving val set the same column names as the train, when the feature is present 
    1. round `mz` and `rt` from `Val` and `Train` 
    2. for each `mz` in `Val`, search for a range on `mzmin` and `mzmax` on train that fits. The `mz_val` need to be between `mzmin_train` and `mzmax_train` 
    3. If a match is found, for each `rt`,`rtmin` and `rtmax` on `Val` search for a range on `rtmin` and `rtmax` on `Train` that fits. The `rt` values need to be between `rtmin_train` and `rtmax_train`. The `rtmin` and `rtmax` from `Val` are used in this case because ocasionally, the range on `Val` or train is too big (big difference in `rt` between samples)
    4. if a match, take the feature name from `Train` and apply on the match
    
**With the features names created:**

3. Features on `Train` and `Val` are ordered 
4. Duplicates are deleted based on the `npeaks` columnn
5. Features that were observed in `Val` but no correspondence was found in `Train` have names filled with `nan`. These are deleted.
4. Features that are on `Train`and were not found in `Val` are added to `Val` and filled with zero (no presence of that feature)
 
 
**To fix: **
 The code for the feature correspondence is not optimized. 
 - After the match with `mz`, the loop searches on the whole dataset for a match in `rt`. This takes more computation, unecessary. 
 - If there is a match of two features, the last one is kept. Could keep both, filter later? 
 


## Feature reference creation - train set

In [4]:
# train is loaded using the function to create the feature names - feature names are created using mz and rt.  
mikania_train = feature_name_creation('5. Gridsearch/mikania_train_processing.csv').reset_index(drop=True)

In [5]:
mikania_train.head()

,features,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks,NEG_GROUP,POS_GROUP,...,ML9_3_mik,ML96_1_mik,ML96_2_mik,ML96_3_mik,ML97_1_mik,ML97_2_mik,ML97_3_mik,ML99_1_mik,ML99_2_mik,ML99_3_mik
0,101_572.8,101,101.138345,102.117217,572.8,439.156,574.441,673,315,330,...,4.679175e+06,5.973447e+06,4.515604e+06,6.924016e+06,3.737724e+06,6.793684e+06,3.589445e+06,4.982682e+06,4.943601e+06,4.584954e+06
1,103_597.4,103,103.151596,104.077694,597.4,471.133,598.699,355,169,182,...,3.533462e+06,4.417315e+06,4.068181e+06,4.132018e+06,3.667740e+06,4.952572e+06,4.515996e+06,3.845193e+06,5.086722e+06,4.798596e+06
2,111_45.2,111,110.639401,111.636593,45.2,39.105,81.512,303,187,97,...,1.627317e+07,1.197850e+07,9.300417e+06,1.227454e+07,8.721469e+06,8.164535e+06,8.876116e+06,8.547570e+06,8.042970e+06,7.753815e+06
3,113_575.3,113,112.639963,113.632866,575.3,551.499,599.104,542,237,255,...,5.238349e+06,6.322030e+06,5.794962e+06,3.939804e+06,5.070993e+06,3.642559e+06,4.522164e+06,6.095111e+06,5.914708e+06,5.225424e+06
4,115_46.5,115,114.639109,115.528369,46.5,42.940,78.906,471,213,244,...,8.146212e+06,3.166693e+07,4.733379e+07,4.567752e+07,2.147885e+07,4.579079e+07,3.924544e+07,3.450197e+07,3.296101e+07,3.384578e+07


## Loading validation val set

In [6]:
# val will be loaded using regular read_csv - the names of the features will come based on comparison
mikania_val = pd.read_csv('5. Gridsearch/mikania_validation_processing.csv',index_col=[0]).reset_index(drop=True).drop(['isotopes', 'adduct','pcgroup'], axis=1) #'npeaks','NEG_GROUP', 'POS_GROUP',

## Rounding mz and rt

In [7]:
# rouding all mz and all rt
mikania_val = rounder(mikania_val)
mikania_train = rounder(mikania_train)

In [8]:
display(mikania_val.iloc[:,0:7].head())
display(mikania_train.iloc[:,0:7].head())

,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks
0,101,101,102,572.8,522.2,574.1,159
1,103,103,104,597.4,594.3,598.7,93
2,111,111,112,46.1,39.4,66.6,80
3,113,113,113,575.0,574.1,598.4,116
4,115,115,115,47.2,43.9,93.1,112


,features,mz,mzmin,mzmax,rt,rtmin,rtmax
0,101_572.8,101,101,102,572.8,439.2,574.4
1,103_597.4,103,103,104,597.4,471.1,598.7
2,111_45.2,111,111,112,45.2,39.1,81.5
3,113_575.3,113,113,114,575.3,551.5,599.1
4,115_46.5,115,115,116,46.5,42.9,78.9


## Feature creation and correspondance on val set

In [9]:
# creating the column
mikania_val['features'] = np.nan

In [10]:
# loop over mikania_val items. 
# Each mz will be tested against all mzmin and mzmax range from train. 
# if in range, test for rt.
# if in range, use the same feature name from train

mikania_val = mikania_val.sort_values('npeaks', ascending=False,ignore_index=True)
mikania_train = mikania_train.sort_values('npeaks', ascending=False,ignore_index=True)

for i in range(len(mikania_val)):
    for j in range(len(mikania_train)):


        if ((mikania_val.loc[i,'mz'] <= mikania_train.loc[j,'mzmax']) 
              & (mikania_val.loc[i,'mz'] >= mikania_train.loc[j,'mzmin'])):
            
            #maybe subset mikania train and then perform things on the subset? 
            
            if (
                ((mikania_val.loc[i,'rt'] <= mikania_train.loc[j,'rtmax']) 
                  & (mikania_val.loc[i,'rt'] >= mikania_train.loc[j,'rtmin'])) or
            
               ((mikania_val.loc[i,'rtmin'] <= mikania_train.loc[j,'rtmax']) 
                  & (mikania_val.loc[i,'rtmin'] >= mikania_train.loc[j,'rtmin'])) or
                
               ((mikania_val.loc[i,'rtmax'] <= mikania_train.loc[j,'rtmax']) 
                & (mikania_val.loc[i,'rtmax'] >= mikania_train.loc[j,'rtmin']))
            ):
                
                mikania_val.loc[i,'features'] = mikania_train.loc[j,'features']
            break

In [11]:
mikania_val

,mz,mzmin,mzmax,rt,rtmin,rtmax,npeaks,NEG_GROUP,POS_GROUP,AQ16_1_mik,...,ML89_1_mik,ML89_2_mik,ML89_3_mik,ML90_1_mik,ML90_2_mik,ML90_3_mik,ML93_1_mik,ML93_2_mik,ML93_3_mik,features
0,117,117,117,572.4,521.2,573.3,372,96,96,5.887108e+07,...,6.333137e+07,6.646837e+07,7.123409e+07,6.464631e+07,6.473378e+07,6.301025e+07,4.236123e+07,4.446527e+07,3.973012e+07,117_572.4
1,302,301,302,547.3,473.8,575.4,302,48,96,2.381940e+06,...,3.372525e+07,3.535800e+07,3.302558e+07,4.497739e+07,5.013374e+07,4.915096e+07,7.694642e+06,1.155288e+07,1.015155e+07,302_547.3
2,217,217,218,572.4,518.6,573.3,241,95,85,1.289468e+07,...,1.512042e+07,1.590213e+07,1.379746e+07,1.467889e+07,1.396555e+07,1.519848e+07,8.829465e+06,7.136288e+06,6.655386e+06,217_572.8
3,402,401,402,547.1,521.1,573.7,234,51,94,2.144783e+06,...,3.802344e+07,3.635515e+07,3.162709e+07,4.257522e+07,4.005340e+07,3.648322e+07,1.217898e+07,1.219293e+07,1.279159e+07,402_547.0
4,488,488,489,348.2,278.3,375.0,213,8,93,4.323549e+06,...,1.779109e+07,2.757322e+07,1.314762e+08,2.092853e+07,1.326263e+08,3.254040e+07,3.582074e+07,3.951186e+07,3.728830e+07,488_348.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,674,674,674,247.0,182.1,249.2,32,0,32,2.813438e+06,...,1.141661e+07,9.989287e+06,8.737974e+06,9.431100e+06,8.432869e+06,9.401396e+06,1.155076e+07,1.163606e+07,1.318347e+07,674_247.9
153,534,533,534,351.2,347.5,354.6,32,29,3,5.578477e+06,...,6.380742e+06,6.812028e+06,5.918371e+06,6.090338e+06,5.022471e+06,5.476122e+06,6.344386e+06,5.182809e+06,5.825212e+06,NaN
154,732,732,733,374.5,265.7,378.7,31,1,29,3.011137e+06,...,2.848826e+06,4.123351e+06,3.476350e+06,4.271722e+06,3.770071e+06,3.580085e+06,8.078104e+06,9.116588e+06,5.907386e+06,NaN
155,654,654,654,246.8,183.6,249.2,30,0,30,8.240939e+05,...,1.977775e+07,1.941051e+07,1.573302e+07,1.536999e+07,1.316865e+07,1.340171e+07,1.200280e+07,1.327482e+07,1.190865e+07,654_247.9


In [12]:
# the process can create duplicates, so removing them is necessary
# the removal is based on the npeaks column. The feature with more npeaks, is kept.
mikania_val = mikania_val.sort_values('npeaks', ascending=False).drop_duplicates('features').sort_index()

# dropping unnecessary columns
mikania_val = mikania_val.drop(['mz', 'mzmin', 'mzmax', 'rt', 
                                  'rtmin', 'rtmax', 'npeaks','NEG_GROUP', 'POS_GROUP'], axis=1)

# removing the duplicates that might arise with the train is also necessary
# drop possible duplicates for train as well
mikania_train = mikania_train.sort_values('npeaks', ascending=False).drop_duplicates('features').sort_index()

# dropping unnecessary  columns
mikania_train = mikania_train.drop(['mz', 'mzmin', 'mzmax', 'rt', 
                                      'rtmin', 'rtmax', 'npeaks','NEG_GROUP', 'POS_GROUP'], axis=1)

# val set might have some feature that don't fit in any range - their feature names will be nan, so need to remove
# train might have some features that wont appear in the val. So, create them in val and set them to zero. 
# first, set index on both to be the features, so its possible to do that.
mikania_train= mikania_train.set_index('features')
mikania_val = mikania_val.dropna().set_index('features') # dropping na and making feature as index

# set method to get the set of index values that are unique 
# subtracting the sets to get the different indexes. 
# concat method to concatenate train and val
# filling the missing values on the concatenation with 0 using the fillna method.

unique_indexes = list(set(mikania_train.index) - set(mikania_val.index))
mikania_val = pd.concat([mikania_val, pd.DataFrame(index=unique_indexes, columns=mikania_val.columns)], sort=True).fillna(0)

# order both val and train features equally
# sort the features - the model needs them at the same sequence
mikania_train = mikania_train.reset_index().sort_values(by='features')
mikania_val = mikania_val.reset_index().sort_values(by='index')




## Bring the class data column

In [13]:
# load
classes_train = pd.read_csv('classes_train_mikania.csv', index_col=[0])
classes_val = pd.read_csv('classes_val_mikania.csv', index_col=[0])

# unite
mikania_train = mikania_train.set_index('features').T.join(classes_train)
display(mikania_train.head())

mikania_val = mikania_val.set_index('index').T.join(classes_val)
display(mikania_val.head())

,1000_338.1,101_572.8,103_597.4,111_45.2,113_575.3,115_46.5,117_572.4,119_337.1,119_41.3,120_573.2,...,904_381.6,918_353.8,920_360.1,923_330.0,940_376.4,946_391.7,962_367.5,977_346.3,988_298.7,class
AQ1_1_mik,1.232216e+06,4.786491e+06,4.266050e+06,2.687591e+07,4.041009e+06,4.642336e+07,6.850176e+07,9.984969e+05,5.405292e+06,5.419716e+06,...,2.757551e+06,2.388167e+06,3.581841e+06,4.042580e+06,1.447280e+06,3.062362e+06,4.357708e+06,4.917440e+06,4.760602e+06,0
AQ1_2_mik,1.162652e+06,4.263014e+06,4.774577e+06,2.426403e+07,3.714576e+06,4.096983e+07,7.254859e+07,5.350251e+05,5.449906e+06,5.997642e+06,...,3.312367e+06,6.569750e+05,2.240067e+06,3.029329e+06,1.305365e+06,2.793934e+06,2.827042e+06,3.798600e+06,5.098263e+06,0
AQ1_3_mik,9.430519e+05,4.099495e+06,4.922569e+06,2.820120e+07,4.087074e+06,4.111136e+07,6.944187e+07,8.729091e+05,6.331146e+06,5.484822e+06,...,1.878620e+06,6.671546e+05,2.133046e+06,3.500603e+06,2.377893e+06,1.644458e+06,2.195071e+06,3.278922e+06,4.824396e+06,0
AQ10_1_mik,8.387890e+05,5.558673e+06,5.206263e+06,2.284447e+07,3.217059e+06,5.631654e+07,6.266933e+07,6.460664e+05,4.208421e+06,5.743904e+06,...,2.320033e+06,7.841861e+05,3.029086e+06,2.734146e+06,2.606019e+06,2.125239e+06,3.100023e+06,3.922228e+06,5.367547e+06,0
AQ10_2_mik,3.547501e+05,3.597790e+06,7.754229e+06,2.075370e+07,5.482342e+06,5.905479e+07,6.384663e+07,1.160967e+06,4.902441e+06,6.034946e+06,...,1.632448e+06,2.016960e+06,2.694035e+06,2.459825e+06,1.558328e+06,1.833980e+06,3.401275e+06,5.435805e+06,4.726148e+06,0


,1000_338.1,101_572.8,103_597.4,111_45.2,113_575.3,115_46.5,117_572.4,119_337.1,119_41.3,120_573.2,...,904_381.6,918_353.8,920_360.1,923_330.0,940_376.4,946_391.7,962_367.5,977_346.3,988_298.7,class
AQ16_1_mik,9.604084e+05,3.969311e+06,3.983464e+06,2.521694e+07,4.548232e+06,3.400086e+07,5.887108e+07,0.0,6.644233e+06,0.0,...,3.372461e+06,3.045742e+06,2.074339e+06,2.590667e+06,3.154463e+06,2.982511e+06,3.286935e+06,3.835976e+06,4.617678e+06,0
AQ16_2_mik,1.173153e+06,4.757843e+06,4.461309e+06,2.536091e+07,4.471601e+06,3.529701e+07,5.996288e+07,0.0,6.761089e+06,0.0,...,3.412054e+06,2.805527e+06,2.407746e+06,3.218943e+06,2.476195e+06,2.075340e+06,2.955413e+06,3.517419e+06,5.412190e+06,0
AQ16_3_mik,2.947566e+05,4.716249e+06,6.176580e+06,2.548268e+07,4.950501e+06,3.050518e+07,6.126746e+07,0.0,6.254150e+06,0.0,...,1.789400e+06,2.700694e+06,1.068769e+06,4.269818e+06,4.598138e+06,3.277033e+06,3.582061e+06,3.547674e+06,3.695863e+06,0
AQ25_1_mik,1.255573e+06,5.292930e+06,4.066484e+06,2.017903e+07,6.947826e+06,3.854644e+07,4.245207e+07,0.0,5.351820e+06,0.0,...,2.276597e+06,2.322428e+06,2.455301e+06,3.411034e+06,4.039373e+06,2.914401e+06,2.695155e+06,3.843059e+06,3.554007e+06,0
AQ25_2_mik,6.181852e+05,5.197629e+06,4.298666e+06,2.031424e+07,6.087268e+06,3.627392e+07,4.825577e+07,0.0,6.368848e+06,0.0,...,4.171824e+06,3.115454e+06,2.394187e+06,3.744213e+06,3.469823e+06,3.784061e+06,3.426345e+06,3.781479e+06,2.319854e+06,0


Data is now ready for ANY machine learning process

# Machine learning

## X y split

In [14]:
X_train = mikania_train.drop("class", axis=1)
y_train = mikania_train["class"]

X_val = mikania_val.drop("class", axis=1)
y_val = mikania_val["class"]

## Training

In [15]:
# https://stackoverflow.com/questions/31948879/using-explicit-predefined-validation-set-for-grid-search-with-sklearn
# https://stackoverflow.com/questions/48390601/explicitly-specifying-test-train-sets-in-gridsearchcv

# Create a list of indices for the training and validation sets
train_indices = np.ones(len(X_train))
val_indices = np.zeros(len(X_val))
cv_indices = np.concatenate((train_indices, val_indices))

# model
svm = SVC()
rf = RandomForestClassifier(random_state=2187)
knn = KNeighborsClassifier()

# params of each model

param_svm = {}
param_svm['model'] = [svm]
param_svm['model__kernel'] = ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
param_svm['model__C'] = [1, 0.9]
param_svm['model__kernel'] = ['rbf']

param_rf = {}
param_rf['model'] = [rf]
param_rf['model__max_depth'] = [10,15]
param_rf['model__n_estimators'] = [100,200,300]
param_rf['model__criterion'] = ['gini', 'entropy']


param_knn = {}
param_knn['model'] = [knn]
param_knn['model__n_neighbors'] = [5,15,25]


# uniting param to test in gridsearch

params_gridsearch = [param_svm,param_rf,param_knn]

# no need to encode or transform data. All is numeric and same scale

# pipe - starts with svm 
pipe = Pipeline([('model', svm)])

cv = PredefinedSplit(cv_indices)

# gridsearch 
grid = GridSearchCV(pipe, params_gridsearch, 
                    cv = cv,
                   scoring = ['f1','matthews_corrcoef'],
                   return_train_score = True, 
                   refit = 'matthews_corrcoef',
                   verbose = 3)

In [16]:
grid.fit(np.vstack((X_train, X_val)), np.hstack((y_train, y_val)))


Fitting 2 folds for each of 17 candidates, totalling 34 fits
[CV 1/2] END model=SVC(), model__C=1, model__kernel=rbf; f1: (train=0.979, test=0.984) matthews_corrcoef: (train=0.958, test=0.969) total time=   0.0s
[CV 2/2] END model=SVC(), model__C=1, model__kernel=rbf; f1: (train=0.984, test=0.951) matthews_corrcoef: (train=0.969, test=0.904) total time=   0.0s
[CV 1/2] END model=SVC(), model__C=0.9, model__kernel=rbf; f1: (train=0.973, test=0.984) matthews_corrcoef: (train=0.945, test=0.969) total time=   0.0s
[CV 2/2] END model=SVC(), model__C=0.9, model__kernel=rbf; f1: (train=0.984, test=0.951) matthews_corrcoef: (train=0.969, test=0.904) total time=   0.0s
[CV 1/2] END model=RandomForestClassifier(random_state=2187), model__criterion=gini, model__max_depth=10, model__n_estimators=100; f1: (train=1.000, test=0.984) matthews_corrcoef: (train=1.000, test=0.969) total time=   0.7s
[CV 2/2] END model=RandomForestClassifier(random_state=2187), model__criterion=gini, model__max_depth=10, 

GridSearchCV(cv=PredefinedSplit(test_fold=array([1, 1, ..., 0, 0])),
             estimator=Pipeline(steps=[('model', SVC())]),
             param_grid=[{'model': [SVC()], 'model__C': [1, 0.9],
                          'model__kernel': ['rbf']},
                         {'model': [RandomForestClassifier(random_state=2187)],
                          'model__criterion': ['gini', 'entropy'],
                          'model__max_depth': [10, 15],
                          'model__n_estimators': [100, 200, 300]},
                         {'model': [KNeighborsClassifier()],
                          'model__n_neighbors': [5, 15, 25]}],
             refit='matthews_corrcoef', return_train_score=True,
             scoring=['f1', 'matthews_corrcoef'], verbose=3)

In [17]:
pd.DataFrame(grid.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model,param_model__C,param_model__kernel,param_model__criterion,param_model__max_depth,param_model__n_estimators,...,std_train_f1,split0_test_matthews_corrcoef,split1_test_matthews_corrcoef,mean_test_matthews_corrcoef,std_test_matthews_corrcoef,rank_test_matthews_corrcoef,split0_train_matthews_corrcoef,split1_train_matthews_corrcoef,mean_train_matthews_corrcoef,std_train_matthews_corrcoef
0,0.005982,4.015088e-03,0.007158,0.000849,SVC(),1,rbf,NaN,NaN,NaN,...,0.002507,0.969223,0.904017,0.936620,0.032603,13,0.958346,0.969223,0.963785,0.005439
1,0.005485,3.491759e-03,0.006981,0.000997,SVC(),0.9,rbf,NaN,NaN,NaN,...,0.005718,0.969223,0.904017,0.936620,0.032603,13,0.945316,0.969223,0.957270,0.011954
2,0.488974,3.298380e-01,0.010461,0.001480,RandomForestClassifier(random_state=2187),NaN,NaN,gini,10,100,...,0.000000,0.969223,0.915432,0.942328,0.026895,3,1.000000,1.000000,1.000000,0.000000
3,0.950544,6.323633e-01,0.016939,0.002976,RandomForestClassifier(random_state=2187),NaN,NaN,gini,10,200,...,0.000000,0.969223,0.909969,0.939596,0.029627,7,1.000000,1.000000,1.000000,0.000000
4,1.417396,9.352663e-01,0.025446,0.003477,RandomForestClassifier(random_state=2187),NaN,NaN,gini,10,300,...,0.000000,0.969223,0.904533,0.936878,0.032345,11,1.000000,1.000000,1.000000,0.000000
5,0.477185,3.206031e-01,0.009974,0.000996,RandomForestClassifier(random_state=2187),NaN,NaN,gini,15,100,...,0.000000,0.969223,0.915432,0.942328,0.026895,3,1.000000,1.000000,1.000000,0.000000
6,0.966431,6.403027e-01,0.017967,0.002977,RandomForestClassifier(random_state=2187),NaN,NaN,gini,15,200,...,0.000000,0.969223,0.909969,0.939596,0.029627,7,1.000000,1.000000,1.000000,0.000000
7,1.457552,9.633558e-01,0.026430,0.002493,RandomForestClassifier(random_state=2187),NaN,NaN,gini,15,300,...,0.000000,0.969223,0.904533,0.936878,0.032345,11,1.000000,1.000000,1.000000,0.000000
8,0.437466,2.708845e-01,0.010970,0.000998,RandomForestClassifier(random_state=2187),NaN,NaN,entropy,10,100,...,0.000000,0.969223,0.920649,0.944936,0.024287,1,1.000000,1.000000,1.000000,0.000000
9,0.892875,5.358572e-01,0.018457,0.002516,RandomForestClassifier(random_state=2187),NaN,NaN,entropy,10,200,...,0.000000,0.969223,0.915183,0.942203,0.027020,5,1.000000,1.000000,1.000000,0.000000


In [18]:
grid.best_params_

{'model': RandomForestClassifier(random_state=2187),
 'model__criterion': 'entropy',
 'model__max_depth': 10,
 'model__n_estimators': 100}

In [19]:
grid.best_score_

0.9449359678336603

In [20]:
round(grid.best_score_,3)

0.945

In [21]:
grid.best_estimator_

Pipeline(steps=[('model',
                 RandomForestClassifier(criterion='entropy', max_depth=10,
                                        random_state=2187))])

In [22]:
results = pd.DataFrame(grid.cv_results_)

results[['param_model','mean_train_matthews_corrcoef','mean_test_matthews_corrcoef']].groupby('param_model',sort=False).mean(['mean_train_matthews_corrcoef','mean_test_matthews_corrcoef']).round(3)

,mean_train_matthews_corrcoef,mean_test_matthews_corrcoef
param_model,,
SVC(),0.961,0.937
RandomForestClassifier(random_state=2187),1.000,0.941
KNeighborsClassifier(),0.952,0.920


In [23]:
results[['param_model','mean_train_f1','mean_test_f1']].groupby('param_model',sort=False).mean(['mean_train_f1','mean_test_f1']).round(3)

,mean_train_f1,mean_test_f1
param_model,,
SVC(),0.980,0.968
RandomForestClassifier(random_state=2187),1.000,0.969
KNeighborsClassifier(),0.975,0.959


In [24]:
# Scores to present

results[['param_model','mean_test_f1','std_test_f1','mean_test_matthews_corrcoef','std_test_matthews_corrcoef']].groupby('param_model',sort=False).mean(['mean_test_f1','std_test_f1','mean_test_matthews_corrcoef','std_test_matthews_corrcoef']).round(3)

,mean_test_f1,std_test_f1,mean_test_matthews_corrcoef,std_test_matthews_corrcoef
param_model,,,,
SVC(),0.968,0.017,0.937,0.033
RandomForestClassifier(random_state=2187),0.969,0.015,0.941,0.028
KNeighborsClassifier(),0.959,0.009,0.920,0.019
